In [38]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
words = open('names.txt', 'r').read().splitlines()


In [40]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [41]:
block_size = 3
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context) 
            Y.append(ix)
            context = context[1:] + [ix]  # crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [42]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [43]:
# MLP revisited
n_embed = 10 # embedding dimension
n_hidden = 200 # hidden layer size
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((len(stoi), n_embed), generator=g)
W1 = torch.randn((block_size * n_embed, n_hidden), generator=g) * ((5/3) / ((n_embed*block_size)**0.5))   # * 0.2
#  = torch.randn(n_hidden, generator=g) * 0.01
W2 = torch.randn((n_hidden, len(stoi)), generator=g) * 0.01
b2 = torch.randn(len(stoi), generator=g) * 0.0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [ ]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # Forward pass
    emb = C[Xb] # batch_size, context_size, embedding_size
    embcat = emb.view(emb.shape[0], -1) 
    hpreact = embcat @ W1 + b1 # batch_size, n_hidden
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias

    # Update running mean and std
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi


    h = torch.tanh(hpreact) # batch_size, n_hidden
    logits = h @ W2 + b2 # batch_size, num_classes
    loss = F.cross_entropy(logits, Yb)

    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()


    # Update
    lr = 0.1 if i < 100000 else 0.01

    for p in parameters:
        p.data += -lr * p.grad
    
    if i % 10000 == 0:
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())



      0/ 200000: 3.3239
  10000/ 200000: 2.0322


In [ ]:
plt.plot(lossi)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(h.abs() > 0.99, cmap='gray', interpolation='nearest')

In [ ]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]
    emb = C[x] # batch_size, context_size, embedding_size
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + # b1 # batch_size, n_hidden
    # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
    hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
    h = torch.tanh(hpreact) # batch_size, n_hidden
    logits = h @ W2 + b2 # batch_size, num_classes
    loss = F.cross_entropy(logits, y)
    print(f"{split} loss: {loss.item():.4f}")
split_loss('train')
split_loss('val')

In [ ]:
# sample from the model
context_size = 3
g = torch.Generator().manual_seed(2147483647 + 10)
for _ in range(20):
    out = []
    context = [0] * context_size
    while True:
        emb = C[torch.tensor([context])] # 1, context_size, embedding_size
        embcat = emb.view(1, -1) 
        h = torch.tanh(embcat @ W1) # 1, n_hidden
        logits = h @ W2 + b2 # 1, num_classes
        probs = F.softmax(logits, dim=1) # 1, num_classes
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        if ix == 0:
            break
        out.append(itos[ix])
    print(''.join(out)) 

In [ ]:
torch.tensor(1/27.0).log()

In [ ]:
# 4-dimensional example of the issue
logits = torch.tensor([0.0, 0.0, 0.0, 0.0])
probs = torch.softmax(logits, dim=0)
loss = -probs[2].log()
probs, loss